# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 79 new papers today
          32 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/31 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.12904


extracting tarball to tmp_2209.12904...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.12906


extracting tarball to tmp_2209.12906...

...

extracting tarball to tmp_2209.13412...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.13449


extracting tarball to tmp_2209.13449...

 done.


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.13449/paper.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'imbalance.tex' from 'tmp_2209.13449/imbalance.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'abundance.tex' from 'tmp_2209.13449/abundance.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/arx

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.12906-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.12906) | **Euclid preparation. XXV. The Euclid Morphology Challenge -- Towards  model-fitting photometry for billions of galaxies**  |
|| Euclid Collaboration, et al. -- incl., <mark>D. Hernández-Lang</mark> |
|*Appeared on*| *2022-09-28*|
|*Comments*| *29 pages, 33 figures. Euclid pre-launch key paper. Companion paper: Bretonniere et al. 2022*|
|**Abstract**| The ESA Euclid mission will provide high-quality imaging for about 1.5 billion galaxies. A software pipeline to automatically process and analyse such a huge amount of data in real time is being developed by the Science Ground Segment of the Euclid Consortium; this pipeline will include a model-fitting algorithm, which will provide photometric and morphological estimates of paramount importance for the core science goals of the mission and for legacy science. The Euclid Morphology Challenge is a comparative investigation of the performance of five model-fitting software packages on simulated Euclid data, aimed at providing the baseline to identify the best suited algorithm to be implemented in the pipeline. In this paper we describe the simulated data set, and we discuss the photometry results. A companion paper (Euclid Collaboration: Bretonni\`ere et al. 2022) is focused on the structural and morphological estimates. We created mock Euclid images simulating five fields of view of 0.48 deg2 each in the $I_E$ band of the VIS instrument, each with three realisations of galaxy profiles (single and double S\'ersic, and 'realistic' profiles obtained with a neural network); for one of the fields in the double S\'ersic realisation, we also simulated images for the three near-infrared $Y_E$, $J_E$ and $H_E$ bands of the NISP-P instrument, and five Rubin/LSST optical complementary bands ($u$, $g$, $r$, $i$, and $z$). To analyse the results we created diagnostic plots and defined ad-hoc metrics. Five model-fitting software packages (DeepLeGATo, Galapagos-2, Morfometryka, ProFit, and SourceXtractor++) were compared, all typically providing good results. (cut) |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.13336-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.13336) | **Machine learning-accelerated chemistry modeling of protoplanetary disks**  |
|| <mark>Grigorii V. Smirnov-Pinchukov</mark>, et al. -- incl., <mark>Dmitry A. Semenov</mark>, <mark>Sierk van Terwisga</mark>, <mark>Thomas Henning</mark> |
|*Appeared on*| *2022-09-28*|
|*Comments*| *11 pages, 5 figures*|
|**Abstract**| Aims. With the large amount of molecular emission data from (sub)millimeter observatories and incoming James Webb Space Telescope infrared spectroscopy, access to fast forward models of the chemical composition of protoplanetary disks is of paramount importance. Methods. We used a thermo-chemical modeling code to generate a diverse population of protoplanetary disk models. We trained a K-nearest neighbors (KNN) regressor to instantly predict the chemistry of other disk models. Results. We show that it is possible to accurately reproduce chemistry using just a small subset of physical conditions, thanks to correlations between the local physical conditions in adopted protoplanetary disk models. We discuss the uncertainties and limitations of this method. Conclusions. The proposed method can be used for Bayesian fitting of the line emission data to retrieve disk properties from observations. We present a pipeline for reproducing the same approach on other disk chemical model sets. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.13302-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.13302) | **Stellar population of the Rosette Nebula and NGC 2244: application of  the probabilistic random forest**  |
|| Koraljka Muzic, et al. -- incl., <mark>Herve Bouy</mark> |
|*Appeared on*| *2022-09-28*|
|*Comments*| *30 pages, 28 figures. Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| (Abridged) In this work, we study the 2.8x2.6 deg2 region in the emblematic Rosette Nebula, centred at the young cluster NGC 2244, with the aim of constructing the most reliable candidate member list to date, determining various structural and kinematic parameters, and learning about the past and the future of the region. Starting from a catalogue containing optical to mid-infrared photometry, as well as positions and proper motions from Gaia EDR3, we apply the Probabilistic Random Forest algorithm and derive membership probability for each source. Based on the list of almost 3000 probable members, of which about a third are concentrated within the radius of 20' from the centre of NGC 2244, we identify various clustered sources and stellar concentrations, and estimate the average distance of 1489+-37 pc (entire region), 1440+-32 pc (NGC 2244) and 1525+-36 pc (NGC 2237). The masses, extinction, and ages are derived by SED fitting, and the internal dynamic is assessed via proper motions relative to the mean proper motion of NGC 2244. NGC 2244 is showing a clear expansion pattern, with an expansion velocity that increases with radius. Its IMF is well represented by two power laws (dN/dM\propto M^{-\alpha}), with slopes \alpha = 1.05+-0.02 for the mass range 0.2 - 1.5 MSun, and \alpha = 2.3+-0.3 for the mass range 1.5 - 20 MSun, in agreement with other star forming regions. The mean age of the region is ~2 Myr. We find evidence for the difference in ages between NGC 2244 and the region associated with the molecular cloud, which appears slightly younger. The velocity dispersion of NGC 2244 is well above the virial velocity dispersion derived from the total mass (1000+-70 MSun) and half-mass radius (3.4+-0.2 pc). From the comparison to other clusters and to numerical simulations, we conclude that NGC 2244 may be unbound, and possibly even formed in a super-virial state. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.13412-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.13412) | **InSight Pressure Data Recalibration, and Its Application to the Study of  Long-Term Pressure Changes on Mars**  |
|| <mark>L. Lange</mark>, et al. |
|*Appeared on*| *2022-09-28*|
|*Comments*| **|
|**Abstract**| Observations of the South Polar Residual Cap suggest a possible erosion of the cap, leading to an increase of the global mass of the atmosphere. We test this assumption by making the first comparison between Viking 1 and InSight surface pressure data, which were recorded 40 years apart. Such a comparison also allows us to determine changes in the dynamics of the seasonal ice caps between these two periods. To do so, we first had to recalibrate the InSight pressure data because of their unexpected sensitivity to the sensor temperature. Then, we had to design a procedure to compare distant pressure measurements. We propose two surface pressure interpolation methods at the local and global scale to do the comparison. The comparison of Viking and InSight seasonal surface pressure variations does not show changes larger than +-8 Pa in the CO2 cycle. Such conclusions are supported by an analysis of Mars Science Laboratory (MSL) pressure data. Further comparisons with images of the south seasonal cap taken by the Viking 2 orbiter and MARCI camera do not display significant changes in the dynamics of this cap over a 40 year period. Only a possible larger extension of the North Cap after the global storm of MY 34 is observed, but the physical mechanisms behind this anomaly are not well determined. Finally, the first comparison of MSL and InSight pressure data suggests a pressure deficit at Gale crater during southern summer, possibly resulting from a large presence of dust suspended within the crater. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.13449-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.13449) | **3D non-LTE iron abundances in FG-type dwarfs**  |
|| A. M. Amarsi, <mark>S. Liljegren</mark>, P. E. Nissen |
|*Appeared on*| *2022-09-28*|
|*Comments*| *17 pages, 11 figures, 5 tables; arXiv abstract abridged; accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| Spectroscopic measurements of iron abundances are prone to systematic modelling errors. We present 3D non-LTE calculations across 32 STAGGER-grid models with effective temperatures from 5000 K to 6500 K, surface gravities of 4.0 dex and 4.5 dex, and metallicities from $-$3 dex to 0 dex, and study the effects on 171 Fe I and 12 Fe II optical lines. In warm metal-poor stars, the 3D non-LTE abundances are up to 0.5 dex larger than 1D LTE abundances inferred from Fe I lines of intermediate excitation potential. In contrast, the 3D non-LTE abundances can be 0.2 dex smaller in cool metal-poor stars when using Fe I lines of low excitation potential. The corresponding abundance differences between 3D non-LTE and 1D non-LTE are generally less severe but can still reach $\pm$0.2 dex. For Fe II lines the 3D abundances range from up to 0.15 dex larger, to 0.10 dex smaller, than 1D abundances, with negligible departures from 3D LTE except for the warmest stars at the lowest metallicities. The results were used to correct 1D LTE abundances of the Sun and Procyon (HD 61421), and of the metal-poor stars HD 84937 and HD 140283, using an interpolation routine based on neural networks. The 3D non-LTE models achieve an improved ionisation balance in all four stars. In the two metal-poor stars, they remove excitation imbalances that amount to 250 K to 300 K errors in effective temperature. For Procyon, the 3D non-LTE models suggest [Fe/H] = 0.11 $\pm$ 0.03, which is significantly larger than literature values based on simpler models. We make the 3D non-LTE interpolation routine for FG-type dwarfs publicly available, in addition to 1D non-LTE departure coefficients for standard MARCS models of FGKM-type dwarfs and giants. These tools, together with an extended 3D LTE grid for Fe II from 2019, can help improve the accuracy of stellar parameter and iron abundance determinations for late-type stars. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.13074-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.13074) | **Euclid preparation: XXII. Selection of Quiescent Galaxies from Mock  Photometry using Machine Learning**  |
|| A.Humphrey, et al. -- incl., <mark>E.Franceschi</mark>, <mark>K.Jahnke</mark>, <mark>S.Ligori</mark> |
|*Appeared on*| *2022-09-28*|
|*Comments*| *37 pages (including appendices), 26 figures; accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| The Euclid Space Telescope will provide deep imaging at optical and near-infrared wavelengths, along with slitless near-infrared spectroscopy, across ~15,000 sq deg of the sky. Euclid is expected to detect ~12 billion astronomical sources, facilitating new insights into cosmology, galaxy evolution, and various other topics. To optimally exploit the expected very large data set, there is the need to develop appropriate methods and software. Here we present a novel machine-learning based methodology for selection of quiescent galaxies using broad-band Euclid I_E, Y_E, J_E, H_E photometry, in combination with multiwavelength photometry from other surveys. The ARIADNE pipeline uses meta-learning to fuse decision-tree ensembles, nearest-neighbours, and deep-learning methods into a single classifier that yields significantly higher accuracy than any of the individual learning methods separately. The pipeline has `sparsity-awareness', so that missing photometry values are still informative for the classification. Our pipeline derives photometric redshifts for galaxies selected as quiescent, aided by the `pseudo-labelling' semi-supervised method. After application of the outlier filter, our pipeline achieves a normalized mean absolute deviation of ~< 0.03 and a fraction of catastrophic outliers of ~< 0.02 when measured against the COSMOS2015 photometric redshifts. We apply our classification pipeline to mock galaxy photometry catalogues corresponding to three main scenarios: (i) Euclid Deep Survey with ancillary ugriz, WISE, and radio data; (ii) Euclid Wide Survey with ancillary ugriz, WISE, and radio data; (iii) Euclid Wide Survey only. Our classification pipeline outperforms UVJ selection, in addition to the Euclid I_E-Y_E, J_E-H_E and u-I_E,I_E-J_E colour-colour methods, with improvements in completeness and the F1-score of up to a factor of 2. (Abridged) |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure .pdf</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.12988-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.12988) | **Polytropic behavior in the structures of Interplanetary Coronal Mass  Ejections**  |
|| <mark>Maher A Dayeh</mark>, <mark>George Livadiotis</mark> |
|*Appeared on*| *2022-09-28*|
|*Comments*| *9 pages, 3 figures*|
|**Abstract**| The polytropic process characterizes the thermodynamics of space plasma particle populations. The polytropic index, ${\gamma}$, is particularly important as it describes the thermodynamic behavior of the system by quantifying the changes in temperature as the system is compressed or expanded. Using Wind spacecraft plasma and magnetic field data during $01/1995 - 12/2018$, we investigate the thermodynamic evolution in 336 Interplanetary Coronal Mass Ejection (ICME) events. For each event, we derive the index ${\gamma}$ in the sheath and magnetic ejecta structures, along with the pre- and post- event regions. We then examine the distributions of all ${\gamma}$ indices in these four regions and derive the entropic gradient of each, which is indicative of the ambient heating. We find that in the ICME sheath region, where wave turbulence is expected to be highest, the thermodynamics takes longest to recover into the original quasi-adiabatic process, while it recovers faster in the quieter ejecta region. This pattern creates a thermodynamic cycle, featuring a near adiabatic value ${\gamma}$ ~ ${\gamma}$${_a}$ (=5/3) upstream of the ICMEs, ${\gamma}$${_a}$ - ${\gamma}$ ~ 0.26 in the sheaths, ${\gamma}$${_a}$ - ${\gamma}$ ~ 0.13 in the ICME ejecta, and recovers again to ${\gamma}$ ~ ${\gamma}$${_a}$ after the passage of the ICME. These results expose the turbulent heating rates in the ICME plasma: the lower the polytropic index from its adiabatic value and closer to its isothermal value, the larger the entropic gradient, and thus, the rate of turbulent heating that heats the ICME plasma. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.13163-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.13163) | **Design and test results of scientific X-ray CMOS cameras**  |
|| Wenxin Wang, et al. -- incl., <mark>Zhixing Ling</mark>, <mark>Chen Zhang</mark>, <mark>Shuang-Nan Zhang</mark> |
|*Appeared on*| *2022-09-28*|
|*Comments*| *7 pages, 5 figures, published by Proc SPIE 2022*|
|**Abstract**| In recent years, scientific CMOS (sCMOS) sensors have found increasing applications to X-ray detection, including X-ray astronomical observations. In order to examine the performance of sCMOS sensors, we have developed X-ray cameras based on sCMOS sensors. Two cameras, CNX22 and CNX 66, have been developed using sCMOS sensors with a photosensitive area of 2 cm * 2 cm and 6 cm * 6 cm, respectively. The designs of the cameras are presented in this paper. The CNX22 camera has a frame rate of 48 fps, whereas CNX66 has a frame rate of currently 20 fps, that can be boosted to 100 fps in the future. The operating temperature of the sCMOS sensor can reach to -20C for CNX22 and -30C for CNX66 with a peltier cooler device. In addition to the commonly used mode of saving original images, the cameras provide a mode of real-time extraction of X-ray events and storage their information, which significantly reduces the requirement for data storage and offline analysis work. For both cameras, the energy resolutions can reach less than 200 eV at 5.9 keV using single-pixel events. These cameras are suitable for X-ray spectroscopy applications in laboratories and calibration for the space X-ray telescopes. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.12906.md
    + _build/html/tmp_2209.12906/./figs/results/totglob_multiband2.png
    + _build/html/tmp_2209.12906/./figs/results/totglob_multiband1_B.png
    + _build/html/tmp_2209.12906/./figs/results/summary_aB.png
    + _build/html/tmp_2209.12906/./figs/results/summary_aD.png
    + _build/html/tmp_2209.12906/./figs/results/summary_aO.png


exported in  _build/html/2209.13336.md
    + _build/html/tmp_2209.13336/./figs/formilium_3par/explanation.png
    + _build/html/tmp_2209.13336/./figs/formilium_3par/HCO+_2d.png
    + _build/html/tmp_2209.13336/./figs/formilium_3par/DCO+_2d.png
    + _build/html/tmp_2209.13336/./figs/formilium_3par/e-_2d.png
    + _build/html/tmp_2209.13336/./figs/formilium_4par/CO_2d.png
    + _build/html/tmp_2209.13336/./figs/formilium_4par/HCO+_2d.png
    + _build/html/tmp_2209.13336/./figs/formilium_4par/DCO+_2d.png
    + _build/html/tmp_2209.13336/./figs/formilium_4par/e-_2d.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\sersics}{S\'ersic }$
$\newcommand{\bts}{{\rm b}/{\rm t} }$
$\newcommand{\bt}{{\rm b}/{\rm t}}$
$\newcommand{\btm}{{\rm b}/{\rm t}}$
$\newcommand{\sersic}{S\'ersic}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\sersics$}{S\'ersic }$
$\newcommand{$\bts$}{{\rm b}/{\rm t} }$
$\newcommand{$\bt$}{{\rm b}/{\rm t}}$
$\newcommand{$\bt$m}{{\rm b}/{\rm t}}$
$\newcommand{$\sersic$}{S\'ersic}$</div>



<div id="title">

#  Morphology Challenge -- Towards model-fitting photometry for billions of galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.12906-b31b1b.svg)](https://arxiv.org/abs/2209.12906)<mark>Appeared on: 2022-09-28</mark> - _29 pages, 33 figures. Euclid pre-launch key paper. Companion paper: Bretonniere et al. 2022_

</div>
<div id="authors">

Euclid Collaboration, et al. -- incl., <mark><mark>D. Hernández-Lang</mark></mark>

</div>
<div id="abstract">

**Abstract:** The European Space Agency's\Euclidmission will provide high-quality imaging for about$1.5$billion galaxies. A software pipeline to automatically process and analyse such a huge amount of data in real time is being developed by the Science Ground Segment of the Euclid Consortium; this pipeline will include a model-fitting algorithm, which will provide photometric and morphological estimates of paramount importance for the core science goals of the mission and for legacy science.   The Euclid Morphology Challenge is a comparative investigation of the performance of five model-fitting software packages on simulated\Eucliddata, aimed at providing the baseline to identify the best suited algorithm to be implemented in the pipeline. In this paper we describe the simulated data set, and we discuss the photometry results. A companion paper (Euclid Collaboration: Bretonni\`ere et al. 2022) is focused on the structural and morphological estimates.   We created mock\Euclidimages simulating five fields of view of 0.48 deg$^2$each in the$I_{\scriptscriptstyle\rm E}$band of the VIS instrument, containing a total of about one and a half million galaxies (of which 350 000 have nominal signal-to-noise ratio above$5$), each with three realisations of galaxy profiles (single and double$\sersic$, and `realistic' profiles obtained with a neural network); for one of the fields in the double$\sersics$realisation, we also simulated images for the three near-infrared$Y_{\scriptscriptstyle\rm E}$,$J_{\scriptscriptstyle\rm E}$and$H_{\scriptscriptstyle\rm E}$bands of the NISP-P instrument, and five Rubin/LSST optical complementary bands ($u$,$g$,$r$,$i$, and$z$), which together form a typical data set for a\Euclidobservation. The images were simulated at the expected\EuclidWide Survey depths. To analyse the results we created diagnostic plots and defined metrics to take into account the completeness of the provided catalogues, and the median biases, dispersions, and outlier fractions of their measured flux distributions.   Five model-fitting software packages (\texttt{DeepLeGATo},\texttt{Galapagos-2},\texttt{Morfometryka},\texttt{ProFit}, and\texttt{SourceXtractor++}) were compared, all typically providing good results. Of the differences among them, some were  at least partly due to the distinct strategies adopted to perform the measurements. In the best case scenario, the median bias of the measured fluxes in the analytical profile realisations is below 1\%at signal-to-noise ratio above 5 in$I_{\scriptscriptstyle\rm E}$, and above 10 in all the other bands; the dispersion of the distribution is typically comparable to the theoretically expected one, with a small fraction of catastrophic outliers. However, we can expect that real observations will prove to be more demanding, since the results were found to be less accurate on the most realistic realisation.   We conclude that existing model-fitting software can provide accurate photometric measurements on\Eucliddata sets. The results of the challenge are fully available and reproducible through an online plotting tool.

</div>

<div id="div_fig1">

<img src="tmp_2209.12906/./figs/results/totglob_multiband2.png" alt="Fig6" width="100%"/>

**Figure 6. -** Same as Fig. \ref{summ1}, for LSST bands. Top to bottom: $u$, $g$, $r$, $i$, $z$. Note the different $y$-- axis scales. (*summ2*)

</div>
<div id="div_fig2">

<img src="tmp_2209.12906/./figs/results/totglob_multiband1_B.png" alt="Fig7" width="100%"/>

**Figure 7. -** Trends for bulges only, in the multi-band DSb4 fit, for $I_{\scriptscriptstyle\rm E}$ and the NIR bands. Left to right: median bias $\mathcal{B}$; dispersion (standard deviation) $\mathcal{D}$; and fraction of outliers $\mathcal{O}$. Top to bottom: $I_{\scriptscriptstyle\rm E}$, $Y_{\scriptscriptstyle\rm E}$, $J_{\scriptscriptstyle\rm E}$, and $H_{\scriptscriptstyle\rm E}$. Each line and colour correspond to a different code as indicated in the legends; note the different $y$-axis scales. (*summ3a*)

</div>
<div id="div_fig3">

<img src="tmp_2209.12906/./figs/results/summary_aB.png" alt="Fig23.1" width="33%"/><img src="tmp_2209.12906/./figs/results/summary_aD.png" alt="Fig23.2" width="33%"/><img src="tmp_2209.12906/./figs/results/summary_aO.png" alt="Fig23.3" width="33%"/>

**Figure 23. -** Visual summary of the diagnostic quantities: (left to right) absolute value of the median bias ($\mathcal{B}$), average ratio of $\sigma_{\rm meas}$ and $\sigma_{\rm true}$($\mathcal{D}$), and outlier fraction ($\mathcal{O}$). In the bias panel, the points corresponding to \texttt{ProFit} DS multi-band runs are not shown, being off the scale (their values are 0.49 and 0.44, respectively). See text for details. (*summaryb*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\hcop}[0]{HCO^+}$
$\newcommand{\dcop}[0]{DCO^+}$
$\newcommand{\hncop}[0]{H^{13}CO^+}$
$\newcommand{\electr}[0]{e^-}$
$\newcommand{\co}[0]{CO}$
$\newcommand{\ico}[0]{^{13}CO}$
$\newcommand{\cio}[0]{C^{18}O}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\hcop$}[0]{HCO^+}$
$\newcommand{$\dcop$}[0]{DCO^+}$
$\newcommand{$\hncop$}[0]{H^{13}CO^+}$
$\newcommand{$\electr$}[0]{e^-}$
$\newcommand{$\co$}[0]{CO}$
$\newcommand{$\ico$}[0]{^{13}CO}$
$\newcommand{$\cio$}[0]{C^{18}O}$</div>



<div id="title">

# Machine learning-accelerated chemistry modeling of protoplanetary disks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.13336-b31b1b.svg)](https://arxiv.org/abs/2209.13336)<mark>Appeared on: 2022-09-28</mark> - _11 pages, 5 figures_

</div>
<div id="authors">

<mark>Grigorii V. Smirnov-Pinchukov</mark>, et al. -- incl., <mark>Dmitry A. Semenov</mark>, <mark>Sierk van Terwisga</mark>, <mark>Thomas Henning</mark>

</div>
<div id="abstract">

**Abstract:** Aims. With the large amount of molecular emission data from (sub)millimeter observatories and incoming James Webb Space Telescope infrared spectroscopy, access to fast forward models of the chemical composition of protoplanetary disks is of paramount importance. Methods. We used a thermo-chemical modeling code to generate a diverse population of protoplanetary disk models. We trained a K-nearest neighbors (KNN) regressor to instantly predict the chemistry of other disk models. Results. We show that it is possible to accurately reproduce chemistry using just a small subset of physical conditions, thanks to correlations between the local physical conditions in adopted protoplanetary disk models. We discuss the uncertainties and limitations of this method. Conclusions. The proposed method can be used for Bayesian fitting of the line emission data to retrieve disk properties from observations. We present a pipeline for reproducing the same approach on other disk chemical model sets. 

</div>

<div id="div_fig1">

<img src="tmp_2209.13336/./figs/formilium_3par/explanation.png" alt="Fig2" width="100%"/>

**Figure 2. -** Performance of ML-accelerated chemistry predictions for CO. (a): mean $\log_{10}$ predicted relative abundance as a function of local temperature, gas density, and ionization rate. Darker areas correspond to larger relative (to H atoms) abundance. (b): median of the difference between the predicted values and test set data (bias, dex), in dex, as a function of temperature and density. Gray areas correspond to an unbiased fit. (c): the standard deviation between the predicted values and test set data, in dex (std, dex). (d): relative density (histogram) of species within the data points, with contours, which are also present on other panels. Various regions of the protoplanetary disk are described on panel (a). A detailed description of the processes leading to this figure is in the main text. Other molecules are shown in Fig. \ref{fig:ions}. (*fig:performance*)

</div>
<div id="div_fig2">

<img src="tmp_2209.13336/./figs/formilium_3par/HCO+_2d.png" alt="Fig3.1" width="33%"/><img src="tmp_2209.13336/./figs/formilium_3par/DCO+_2d.png" alt="Fig3.2" width="33%"/><img src="tmp_2209.13336/./figs/formilium_3par/e-_2d.png" alt="Fig3.3" width="33%"/>

**Figure 3. -** Performance of ML-accelerated chemistry predictions for $\hcop$ , $\dcop$ , and electrons. (*fig:ions*)

</div>
<div id="div_fig3">

<img src="tmp_2209.13336/./figs/formilium_4par/CO_2d.png" alt="Fig4.1" width="25%"/><img src="tmp_2209.13336/./figs/formilium_4par/HCO+_2d.png" alt="Fig4.2" width="25%"/><img src="tmp_2209.13336/./figs/formilium_4par/DCO+_2d.png" alt="Fig4.3" width="25%"/><img src="tmp_2209.13336/./figs/formilium_4par/e-_2d.png" alt="Fig4.4" width="25%"/>

**Figure 4. -** Effect of adding UV radiation strength to the set of input features. The panels show the performance of ML-accelerated chemistry predictions, as in Fig. \ref{fig:performance}. Adding UV radiation strength increases the accuracy of the fit, but only slightly in the areas of the parameter space dominated by the selected molecules. Depending on the molecular species and constraints on calculation time, it is not necessary to use UV as a parameter. (*fig:ions4*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

47  publications in the last 7 days.
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers